# MPLID Statistical Analysis

This notebook provides statistical analysis of the MPLID dataset for the Data Note manuscript.

**Author**: Folorunsho Bright Omage  
**License**: MIT

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Configure plotting for publication
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'

## 1. Load Complete Dataset

In [ ]:
# Load all splits
train = pd.read_csv('../data/processed/train_residues.csv')
val = pd.read_csv('../data/processed/val_residues.csv')
test = pd.read_csv('../data/processed/test_residues.csv')

# Combine for overall statistics
all_data = pd.concat([train, val, test], ignore_index=True)

print("Dataset Summary:")
print(f"  Total proteins: {all_data['pdb_id'].nunique():,}")
print(f"  Total residues: {len(all_data):,}")
print(f"  Contact residues: {all_data['is_contact'].sum():,}")
print(f"  Contact rate: {all_data['is_contact'].mean():.2%}")
print(f"  Sequence clusters: {all_data['cluster_id'].nunique():,}")

## 2. Split Statistics

In [ ]:
# Calculate statistics for each split
split_stats = []

for name, df in [('Train', train), ('Validation', val), ('Test', test)]:
    stats_dict = {
        'Split': name,
        'Proteins': df['pdb_id'].nunique(),
        'Residues': len(df),
        'Contacts': df['is_contact'].sum(),
        'Contact Rate (%)': df['is_contact'].mean() * 100,
        'Clusters': df['cluster_id'].nunique(),
    }
    split_stats.append(stats_dict)

split_df = pd.DataFrame(split_stats)
print(split_df.to_string(index=False))

## 3. Lipid Type Analysis

In [ ]:
# Lipid type distribution
contacts = all_data[all_data['is_contact'] == 1]
lipid_stats = contacts.groupby('lipid_type').agg({
    'pdb_id': 'nunique',
    'is_contact': 'count'
}).rename(columns={'pdb_id': 'n_proteins', 'is_contact': 'n_contacts'})

lipid_stats = lipid_stats.sort_values('n_contacts', ascending=False)
print("Top 20 Lipid Types:")
print(lipid_stats.head(20))

## 4. Publication Figure: Dataset Overview

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# A) Lipid type distribution
ax = axes[0, 0]
top_lipids = lipid_stats.head(10)['n_contacts']
top_lipids.plot(kind='bar', ax=ax, color='steelblue', edgecolor='black')
ax.set_xlabel('Lipid Type')
ax.set_ylabel('Number of Contacts')
ax.set_title('A) Top 10 Lipid Types')
ax.tick_params(axis='x', rotation=45)

# B) Amino acid enrichment
ax = axes[0, 1]
aa_contact = contacts['residue_name'].value_counts()
aa_total = all_data['residue_name'].value_counts()
enrichment = (aa_contact / aa_contact.sum()) / (aa_total / aa_total.sum())
enrichment = enrichment.sort_values(ascending=False)
colors = ['#2E7D32' if v > 1 else '#C62828' for v in enrichment.values]
enrichment.plot(kind='bar', ax=ax, color=colors, edgecolor='black')
ax.axhline(y=1, color='black', linestyle='--', linewidth=1)
ax.set_xlabel('Amino Acid')
ax.set_ylabel('Enrichment Ratio')
ax.set_title('B) Amino Acid Enrichment')

# C) Contact distance distribution
ax = axes[1, 0]
distances = contacts['min_distance'].dropna()
ax.hist(distances, bins=40, color='steelblue', edgecolor='black', alpha=0.7)
ax.axvline(x=4.0, color='red', linestyle='--', linewidth=2)
ax.set_xlabel('Distance to Lipid (Å)')
ax.set_ylabel('Frequency')
ax.set_title('C) Contact Distance Distribution')

# D) Split distribution
ax = axes[1, 1]
split_sizes = [len(train), len(val), len(test)]
split_labels = ['Train', 'Validation', 'Test']
colors = ['#1976D2', '#7CB342', '#FB8C00']
ax.pie(split_sizes, labels=split_labels, colors=colors, autopct='%1.1f%%',
       startangle=90, explode=(0.02, 0.02, 0.02))
ax.set_title('D) Dataset Split Distribution')

plt.tight_layout()
plt.savefig('../figures/dataset_overview.png', dpi=300)
plt.savefig('../figures/dataset_overview.pdf')
plt.show()

## 5. Comparison with Related Datasets

In [ ]:
# Create comparison table
comparison = pd.DataFrame({
    'Dataset': ['DREAMM', 'OPM-derived', 'MPLID'],
    'Proteins': [54, 15096, all_data['pdb_id'].nunique()],
    'Label Source': ['Experimental', 'Computational', 'Experimental'],
    'Public': ['Limited', 'Yes', 'Yes'],
})
print(comparison.to_string(index=False))

## 6. Export Statistics for Manuscript

In [ ]:
# Export key statistics to JSON
import json

manuscript_stats = {
    'total_proteins': int(all_data['pdb_id'].nunique()),
    'total_residues': int(len(all_data)),
    'total_contacts': int(all_data['is_contact'].sum()),
    'contact_rate_percent': round(all_data['is_contact'].mean() * 100, 2),
    'sequence_clusters': int(all_data['cluster_id'].nunique()),
    'lipid_types': int(contacts['lipid_type'].nunique()),
    'splits': {
        'train': {'proteins': int(train['pdb_id'].nunique()), 'residues': int(len(train))},
        'val': {'proteins': int(val['pdb_id'].nunique()), 'residues': int(len(val))},
        'test': {'proteins': int(test['pdb_id'].nunique()), 'residues': int(len(test))},
    }
}

with open('../data/statistics/manuscript_statistics.json', 'w') as f:
    json.dump(manuscript_stats, f, indent=2)

print("Statistics exported to manuscript_statistics.json")